In [1]:
cd

/Users/Martina


In [376]:
pwd

'/Users/Martina'

In [357]:
import pandas as pd
import numpy as np
import random

data = pd.read_csv('Desktop/Interactions/formatted_questions.csv')
data_untouched = pd.read_csv('Desktop/Interactions/formatted_questions.csv')
data = data.drop(['A1', 'answer_start', 'Unnamed: 0', 'category'], axis =1 )

In [358]:
data.head()

,question,context
0,Am I at risk for COVID-19 from a package or pr...,There is still a lot that is unknown about the...
1,Am I likely to get sicker if I'm exposed to mu...,Symptom severity can be influenced by many dif...
2,Are antibiotics effective in preventing or tre...,No. According to the World Health Organization...
3,Are there therapies available to treat COVID-19?,Scientists are currently testing different typ...
4,Are there two strains of the COVID-19 virus?,The existence of an S strain and an L strain r...


In [359]:
len(data)

411

In [360]:
new = pd.DataFrame(columns =['question','context'])

for ques in range(len(data)):   #ques
    for con in range(len(data)):   
        new_row = {'question': data['question'][ques], 'context': data['context'][con]}
        new = new.append(new_row, ignore_index=True)
        


KeyboardInterrupt: 

In [6]:
new['label'] = 0

In [7]:
for i in range(0,168921,412):
    new['label'][i] = 1


<ipython-input-7-1709dc6cec5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['label'][i] = 1


In [9]:
new.to_csv('AnswerSelectionDataLong.csv')

In [361]:
df = pd.read_csv('AnswerSelectionDataLong.csv', index_col = 0)

In [362]:
df.head()

,question,context,label
0,Am I at risk for COVID-19 from a package or pr...,There is still a lot that is unknown about the...,1
1,Am I at risk for COVID-19 from a package or pr...,Symptom severity can be influenced by many dif...,0
2,Am I at risk for COVID-19 from a package or pr...,No. According to the World Health Organization...,0
3,Am I at risk for COVID-19 from a package or pr...,Scientists are currently testing different typ...,0
4,Am I at risk for COVID-19 from a package or pr...,The existence of an S strain and an L strain r...,0


In [363]:
len(new)

4280

In [364]:
new.loc[135219]

KeyError: 135219

In [365]:
train = new[:135219] #approx 80 %

In [366]:
test = new[135219:]

In [21]:
train.to_csv('AnswerSelectionDataLongTrain.csv')
test.to_csv('AnswerSelectionDataLongTest.csv')

In [22]:
train.to_csv('AnswerSelectionDataLongTrain.tsv',index=False, sep="\t")
test.to_csv('AnswerSelectionDataLongTest.tsv',index=False, sep="\t")


In [27]:
new['question'][0:411]

0      Am I at risk for COVID-19 from a package or pr...
1      Am I at risk for COVID-19 from a package or pr...
2      Am I at risk for COVID-19 from a package or pr...
3      Am I at risk for COVID-19 from a package or pr...
4      Am I at risk for COVID-19 from a package or pr...
                             ...                        
406    Am I at risk for COVID-19 from a package or pr...
407    Am I at risk for COVID-19 from a package or pr...
408    Am I at risk for COVID-19 from a package or pr...
409    Am I at risk for COVID-19 from a package or pr...
410    Am I at risk for COVID-19 from a package or pr...
Name: question, Length: 411, dtype: object

In [367]:
df["question"] = df['question'].str.replace('[^\w\s]','').str.lower()
df["context"] = df['context'].str.replace('[^\w\s]','').str.lower()

from sklearn.feature_extraction.text import CountVectorizer


#get the text column 
docs=df['context'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer()
word_count_vector=cv.fit_transform(docs)



from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

doc=df['question'].tolist()


tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))



## context


# Tokenize and generate count vectors
word_vec = df.context.apply(str.split).apply(pd.value_counts).fillna(0)  

# Compute term frequencies
tf_c = word_vec.divide(np.sum(word_vec, axis=1), axis=0)

# Compute inverse document frequencies
idf = 1+(np.log10(len(tf_c) / 1+(word_vec[word_vec > 0].count())))

# Compute TF-IDF vectors
tfidf = np.multiply(tf_c, idf.to_frame().T)

print(tfidf)

## question 

# Tokenize and generate count vectors
word_vecQ = df.question.apply(str.split).apply(pd.value_counts).fillna(0)

# Compute term frequencies
tf_q = word_vecQ.divide(np.sum(word_vecQ, axis=1), axis=0)



AttributeError: 'list' object has no attribute 'lower'

In [ ]:
my_list = [1,2,3,4,5]
my_list[0] = 2

In [ ]:
my_list.insert(0,2)

In [ ]:
my_list

In [ ]:
len(document_list)

In [368]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidfvectorizer = TfidfVectorizer(analyzer='word' , stop_words='english',)
sklearn_tfidf = TfidfVectorizer()


In [369]:
cosine_array = np.empty((0,411),int)
document_list = df.context[0:411].tolist()                       #make all context documents a list
document_list.insert(0,data['question'][0])               #insert first question




In [ ]:
cosine_array = np.empty((0,len(df)),int)
document_list = df.context.tolist()                       #make all context documents a list
document_list.insert(0,data['question'][0])               #insert first question

for row in range(len(df)):
    tfidf_matrix = sklearn_tfidf.fit_transform(document_list)   # fit all documents into TFIDF vector
    temp = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])  #compare first document to all documents
    cosine_array = np.append(cosine_array, temp, axis=0)
    document_list[0] = data['question'][row+1]
print(cosine_array)

In [370]:
len(document_list)

412

In [371]:
df_len = len(df)
for row in range(0,168921,411):
    document_list[0] = df['question'][row]
    tfidf_matrix = sklearn_tfidf.fit_transform(document_list)   # fit all documents into TFIDF vector
    temp = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])  #compare first document to all documents
    cosine_array = np.append(cosine_array, temp, axis=0)
print(cosine_array)


[[0.06670989 0.05561591 0.01227996 ... 0.02160826 0.05011916 0.01802401]
 [0.0371824  0.1707967  0.01555102 ... 0.04832217 0.10599847 0.13024292]
 [0.04847887 0.04368226 0.33848175 ... 0.02697251 0.03927299 0.00678892]
 ...
 [0.05417188 0.02445447 0.04059237 ... 0.01944379 0.04019306 0.00253449]
 [0.         0.         0.0286556  ... 0.06363201 0.03138303 0.        ]
 [0.05142979 0.0201753  0.00920225 ... 0.04189659 0.03577636 0.14258362]]


In [372]:
df

,question,context,label
0,am i at risk for covid19 from a package or pro...,there is still a lot that is unknown about the...,1
1,am i at risk for covid19 from a package or pro...,symptom severity can be influenced by many dif...,0
2,am i at risk for covid19 from a package or pro...,no according to the world health organization ...,0
3,am i at risk for covid19 from a package or pro...,scientists are currently testing different typ...,0
4,am i at risk for covid19 from a package or pro...,the existence of an s strain and an l strain r...,0
...,...,...,...
168916,what happens if there is a sick passenger on a...,cdc recommends all travelers particularly olde...,0
168917,what happens if there is a sick passenger on a...,cdc does not require an autopsy before the rem...,0
168918,what happens if there is a sick passenger on a...,cdc provides recommendations for international...,0
168919,what happens if there is a sick passenger on a...,up to 25 percent of people infected with the n...,0


In [373]:
new_df = pd.DataFrame(columns =['question','context','label','cosine_sim'])


In [315]:
new_df = pd.DataFrame(columns =['question','context','label','cosine_sim'])

for row in range(411):
    largest = df['cosine_sim'][(row*411):((row+1)*411)].astype(float).nlargest(50)
    new_df = new_df.append(df.iloc[largest.index])


,question,context,label,cosine_sim


In [290]:
largest

168920    0.143070
168750    0.133187
168733    0.094270
168833    0.082804
168722    0.075441
168721    0.073539
168523    0.053486
168727    0.051043
168816    0.049629
168817    0.049629
168818    0.049629
168709    0.049057
168710    0.049057
168510    0.047753
168738    0.047191
168690    0.046803
168596    0.046660
168597    0.046660
168598    0.046660
168741    0.046288
168832    0.046202
168524    0.046061
168712    0.044163
168713    0.044163
168686    0.043841
168808    0.042888
168653    0.042098
168522    0.041378
168719    0.041295
168831    0.041284
168918    0.040733
168605    0.039889
168702    0.039815
168703    0.039683
168859    0.038107
168860    0.038107
168541    0.037925
168850    0.037095
168844    0.036495
168845    0.036495
168573    0.036124
168647    0.036101
168648    0.036101
168737    0.036056
168542    0.035863
168919    0.035831
168694    0.034987
168748    0.034842
168811    0.034783
168812    0.034783
Name: cosine_sim, dtype: float64

In [375]:
for row in range(411):
    largest = df['cosine_sim'][(row*411):((row+1)*411)].astype(float).nlargest(50)  #get the largest 50 out of each question
    ct = 0
    for i in largest.index:
        if df['label'][i] ==1:       #make sure ground truth is among the largest
            ct+=1
    if ct == 0:
        new_df = new_df.append(df[(row*411):((row+1)*411)].loc[[1]])
        new_df = new_df.append(df.iloc[largest.index][0:49])
    else: 
        new_df.append(df.iloc[largest.index])

KeyError: 'cosine_sim'

In [355]:
largest.index

Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0],
           dtype='int64', name='label')

In [340]:
df['label']

KeyError: 'label'

In [354]:
for i in df['question'][0:4]:
    print(i)

Am I at risk for COVID-19 from a package or products shipping from China?
Am I at risk for COVID-19 from a package or products shipping from China?
Am I at risk for COVID-19 from a package or products shipping from China?
Am I at risk for COVID-19 from a package or products shipping from China?


In [330]:
df.set_index('label', inplace=True)

In [341]:
df.index

Int64Index([1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
           dtype='int64', name='label', length=168921)

In [334]:
df['question'][0:400]

label
1    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
                           ...                        
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
0    Am I at risk for COVID-19 from a package or pr...
Name: question, Length: 400, dtype: object

In [282]:
ct =0
for i in largest.index:
    if df['label'][i] == 1:
        ct+=1
if ct == 0:
    new_df = new_df.appendd.(df.iloc[df['label'.loc[1]]]) 


yes


In [277]:
ct =0
for i in new_df['label']:
    if i == 1:
        ct+=1

In [278]:
ct

376

In [ ]:
ct = 0
for row in range(len(df)):    
    cosine_list = cosine_array[row].tolist()     #covert array row to list
    sorted_list = sorted(cosine_list)          # sort list
    highest_value = sorted_list[-1]            #get the highest cosine value for the list
    if cosine_list[row] == highest_value:      # check if the true label matches the highest value 
        print(row, highest_value)
        ct +=1

print(ct,"out of",len(cosine_array), "instances where the correct answer was chosen for TRAIN DATA" )
# MRR(train_data,cosine_array,len(train_data))

In [ ]:
cosine_array = np.empty((0,len(df)),int)
document_list = df.context.tolist()                       #make all context documents a list
document_list.insert(0,data['question'][0])               #insert first question

for row in range(len(df)):
    tfidf_matrix = sklearn_tfidf.fit_transform(document_list)   # fit all documents into TFIDF vector
    temp = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])  #compare first document to all documents
    cosine_array = np.append(cosine_array, temp, axis=0)
    document_list[0] = data['question'][row+1]
print(cosine_array)



In [73]:
for i in range(411,168921,411):
    print(i)

411
822
1233
1644
2055
2466
2877
3288
3699
4110
4521
4932
5343
5754
6165
6576
6987
7398
7809
8220
8631
9042
9453
9864
10275
10686
11097
11508
11919
12330
12741
13152
13563
13974
14385
14796
15207
15618
16029
16440
16851
17262
17673
18084
18495
18906
19317
19728
20139
20550
20961
21372
21783
22194
22605
23016
23427
23838
24249
24660
25071
25482
25893
26304
26715
27126
27537
27948
28359
28770
29181
29592
30003
30414
30825
31236
31647
32058
32469
32880
33291
33702
34113
34524
34935
35346
35757
36168
36579
36990
37401
37812
38223
38634
39045
39456
39867
40278
40689
41100
41511
41922
42333
42744
43155
43566
43977
44388
44799
45210
45621
46032
46443
46854
47265
47676
48087
48498
48909
49320
49731
50142
50553
50964
51375
51786
52197
52608
53019
53430
53841
54252
54663
55074
55485
55896
56307
56718
57129
57540
57951
58362
58773
59184
59595
60006
60417
60828
61239
61650
62061
62472
62883
63294
63705
64116
64527
64938
65349
65760
66171
66582
66993
67404
67815
68226
68637
69048
69459
69870
70281


In [ ]:

ct = 0
for row in range(len(train_data)):    
    cosine_list = cosine_array[row].tolist()     #covert array row to list
    sorted_list = sorted(cosine_list)          # sort list
    highest_value = sorted_list[-1]            #get the highest cosine value for the list
    if cosine_list[row] == highest_value:      # check if the true label matches the highest value 
        print(row, highest_value)
        ct +=1

print(ct,"out of",len(cosine_array), "instances where the correct answer was chosen for TRAIN DATA" )
# MRR(train_data,cosine_array,len(train_data))

In [32]:
new['question'][411:822]

411    Am I likely to get sicker if I'm exposed to mu...
412    Am I likely to get sicker if I'm exposed to mu...
413    Am I likely to get sicker if I'm exposed to mu...
414    Am I likely to get sicker if I'm exposed to mu...
415    Am I likely to get sicker if I'm exposed to mu...
                             ...                        
817    Am I likely to get sicker if I'm exposed to mu...
818    Am I likely to get sicker if I'm exposed to mu...
819    Am I likely to get sicker if I'm exposed to mu...
820    Am I likely to get sicker if I'm exposed to mu...
821    Am I likely to get sicker if I'm exposed to mu...
Name: question, Length: 411, dtype: object

In [ ]:
#trial

In [191]:
df = pd.DataFrame(columns=['question','context'])

In [192]:
data_values = {'question':['q1', 'q2', 'q3', 'q4'], 
        'context':['c1', 'c2', 'c3', 'c4']} 

In [193]:
df = pd.DataFrame(data_values)

In [195]:
df

,question,context
0,q1,c1
1,q2,c2
2,q3,c3
3,q4,c4


In [196]:
new_ = pd.DataFrame(columns =['question','context'])

for ques in range(len(df)):   #ques
    for con in range(len(df)):   
        new__row = {'question': df['question'][ques], 'context': df['context'][con]}
        new_ = new_.append(new__row, ignore_index=True)

In [197]:
new_

,question,context
0,q1,c1
1,q1,c2
2,q1,c3
3,q1,c4
4,q2,c1
5,q2,c2
6,q2,c3
7,q2,c4
8,q3,c1
9,q3,c2


In [171]:
new_
new_['label'] = 0
new_

len(new_)

16

In [179]:
for i in range(0,168921,412):
    
    

0
412
824
1236
1648
2060
2472
2884
3296
3708
4120
4532
4944
5356
5768
6180
6592
7004
7416
7828
8240
8652
9064
9476
9888
10300
10712
11124
11536
11948
12360
12772
13184
13596
14008
14420
14832
15244
15656
16068
16480
16892
17304
17716
18128
18540
18952
19364
19776
20188
20600
21012
21424
21836
22248
22660
23072
23484
23896
24308
24720
25132
25544
25956
26368
26780
27192
27604
28016
28428
28840
29252
29664
30076
30488
30900
31312
31724
32136
32548
32960
33372
33784
34196
34608
35020
35432
35844
36256
36668
37080
37492
37904
38316
38728
39140
39552
39964
40376
40788
41200
41612
42024
42436
42848
43260
43672
44084
44496
44908
45320
45732
46144
46556
46968
47380
47792
48204
48616
49028
49440
49852
50264
50676
51088
51500
51912
52324
52736
53148
53560
53972
54384
54796
55208
55620
56032
56444
56856
57268
57680
58092
58504
58916
59328
59740
60152
60564
60976
61388
61800
62212
62624
63036
63448
63860
64272
64684
65096
65508
65920
66332
66744
67156
67568
67980
68392
68804
69216
69628
70040
7045

In [19]:
"""Add rows with Zeros"""
def Add_Zeros(df):
    zeros = np.where(np.empty_like(df.values), 0, 0)
    data = np.hstack([df.values, zeros]).reshape(-1, df.shape[1])
    df_ordered = pd.DataFrame(data, columns=df.columns)
    return df_ordered

new_data = Add_Zeros(data)


In [20]:
new_data

,question,context
0,Am I at risk for COVID-19 from a package or pr...,There is still a lot that is unknown about the...
1,0,0
2,Am I likely to get sicker if I'm exposed to mu...,Symptom severity can be influenced by many dif...
3,0,0
4,Are antibiotics effective in preventing or tre...,No. According to the World Health Organization...
...,...,...
817,0,0
818,Should travelers wear facemasks?,Up to 25 percent of people infected with the n...
819,0,0
820,What happens if there is a sick passenger on a...,"Under current federal regulations, pilots must..."


In [21]:
new_data = Add_Zeros(new_data)

In [22]:
new_data

,question,context
0,Am I at risk for COVID-19 from a package or pr...,There is still a lot that is unknown about the...
1,0,0
2,0,0
3,0,0
4,Am I likely to get sicker if I'm exposed to mu...,Symptom severity can be influenced by many dif...
...,...,...
1639,0,0
1640,What happens if there is a sick passenger on a...,"Under current federal regulations, pilots must..."
1641,0,0
1642,0,0
